In [7]:
from playwright.async_api import async_playwright
import asyncio
import os
import pandas as pd
from pathlib import Path

In [12]:
# Určení proměnných
# url = odkaz na stránky se stažením souborů
# download_path = cesta kam se mají uložit

url = "https://mpo.gov.cz/cz/podnikani/zivnostenske-podnikani/statisticke-udaje-o-podnikatelich/pocty-podnikatelu-dle-obcanstvi-podnikajicich-v-ceske-republice--151024/"
download_path = "./Download"

In [50]:
# Define the Playwright ~ Chromium download function

async def fetch_and_download_files(url, download_path):
    # Make sure the download path exists
    if not os.path.exists(download_path):
        os.makedirs(download_path)
    
    print("Done 0")
    async with async_playwright() as pw:
        browser = await pw.chromium.launch(headless=True)
        page = await browser.new_page()

        # Set the download behavior
        page.on('download', lambda download: download.save_as(os.path.join(download_path, download.suggested_filename)))

        await page.goto(url)
        print("Done 1")

        # Wait for the page to be fully loaded
        await page.wait_for_load_state('load')
        print("Done 2")

        # Wait for the download links to appear
        await page.wait_for_selector('a.download')
        print("Done 3")

        # Get all 'a' elements with the class 'download'
        download_links = await page.query_selector_all('a.download')
        print("Done 4")

        # Click each download link and wait for the file to be downloaded
        if download_links:
            d_counts = 0
            for link in download_links:
                href = await link.get_attribute('href')
                #print(f"Attempting to download: {href}")
                # Click the link to start the download
                await link.click()
                print("Clicking the download link")
                d_counts =+ 1
                # Wait for the download to complete (you can adjust this wait time if necessary)
                await page.wait_for_timeout(500)  # Adjust if needed to allow the download to finish
        else:
            print("No download links found")
        
        print(f"Done with {d_counts} downloads")
        await browser.close()


In [ ]:
# -------------------
# RUN download script
# -------------------
await fetch_and_download_files(url, download_path)

In [19]:
# Get names of files in the Download folder
# Iterate over all files in the folder

folder_path = Path(download_path)

processed = 0
filenames = []
# Iterate over all files in the folder
for file in folder_path.iterdir():
    if file.is_file():  # Check if it's a file
        processed += 1
        filenames.append(file.name)
        #print(f"Processing file: {file.name}")
        # Add your processing logic here

print(f"Ve složce je {processed} souborů.")
print(f"Toto jsou jejich jména: {filenames}")

Ve složce je 17 souborů.
Toto jsou jejich jména: ['2024_3.xlsx', '2019_1.xlsx', '2016_4.xlsx', '2017_1.xlsx', '2024_2.xlsx', '2018_4.xlsx', '2016_2.pdf', '2018_3.xlsx', '2018_2.xlsx', '2017_4.xlsx', '2018_1.xlsx', '2020_1.xlsx', '2017_2.xlsx', '2019_3.xlsx', '2024_1.xlsx', '2019_2.xlsx', '2017_3.xlsx']


In [ ]:
# Open the password-protected file
file_path = (f"{download_path}/2016_4"
df = pd.read_excel(file_path, engine='openpyxl', password=password)

# Save as CSV
csv_path = 'output.csv'
df.to_csv(csv_path, index=False)
print(f"File converted to: {csv_path}")
